<img style="float: right; width: 150px;" src="data/firrm.jpg">

## <span style="color:#4375c7">Quantitative Finance</span>
***
*Course materials are for educational purposes only. Nothing contained herein should be considered investment advice or an opinion regarding the suitability of any security. For more information about this course, please contact us.*
***


# Importing data from the web
So far, you have been importing data from local sources (csv and SQL). Sometimes web services provide an application programming interface (API) that makes it much easier to interact with the specific service. The API includes protocols and routines. Basically, an API is a piece of software that facilitates communication between (other) two pieces of software. Pro tip: Whenever you are thinking about scrapping a website, check if they offer an API.

Sometimes data is not easily available, and you may not be able to download data directly from an API or another download service. If this is the case, you should use web scraping techniques to get some data (BUT only if it is not explicitly forbidden by the website's terms and conditions!) As we move forward in this lecture, we will cover the following topics.


### Session contents:
1. **[Download data from an API](#api)**
2. **[Import and save locally datasets from the web](#import)**
3. **[Web scraping 101](#scrape)**
4. **[Session Takeaways](#takeaway)**
***

## 1. Download data from an API <a id='api'></a>

APIs are quite common for accessing data from the web. In general, you need to take the following steps to download the data you need: Register on the website, get a personal API key, and write a Python script to connect to the API service with your personal key. If you have any problems with the API, read the documentation!

**Warning:** Do not store your API key in your Python file. Why not? Some APIs are not free, so if you save your script on GitHub or in a cloud service, your API key can be misused by third parties. 

Now, let's dive deeper into APIs. Go to [Quandl] (https://docs.quandl.com/) and read the documentation. Sign up and request an API key. Store your key as an environment variable using `os.environ` from the `os` library, and then delete that cell from your notebook. This will ensure that your key is only stored locally. Connect to quandl with your key and make queries. Let's say you are interested in rare metals, e.g, the supply and demand structure of rhodium. Find the database in quandl that offers rare metals data and download the data using `quandl.get()`. Finally, plot the data using pandas.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import quandl

In [ ]:
from api_key_loader import api_key_loader

keyloader = api_key_loader()
print(keyloader.lsapi())

In [ ]:
quandl.ApiConfig.api_key = keyloader.get("quandl")

In [ ]:
data = quandl.get('JOHNMATT/RHODIUM', start_date='2002-01-01')

In [ ]:
plt.rcParams['figure.figsize'] = [16, 6]
data[['Total Supply', 'Total Demand']].plot()
plt.show()

***
Great work so far. Let us consider another source of information: news articles. Go to the [News API](https://newsapi.org/) and get an API key. Save the key locally and request news articles about `bitcoin`. Your request should return a JSON object. Since APIs mostly (99%) return data in JSON format, you need to be familiar with the JSON (JavaScript Object Notation) format. JSON provides a hierarchical data structure similar to the dict data type. 

In [ ]:
from newsapi import NewsApiClient

In [ ]:
newsapi = NewsApiClient(api_key = keyloader.get("newsapi"))

In [ ]:
bitcoin_news = newsapi.get_top_headlines(q='bitcoin')
bitcoin_news

***

### Hands-on

1. Register on quandl and make requests. If available, download a nickel price index and visualize the time series.


2. Register on News API and download headlines about electric vehicles. Take the first article you get and print the content to the console.

***

## 2. Import and locally save datasets from the web <a id='import'></a>

**What is an URL?**


URL stands for Uniform Resource Locator and is used to refer to web resources (like an address referring to your home address).
<br>
Igredients:
- Protocol identifier: http, https, File Transfer Protocol (FTP), Gopher, File, and News
- Resource name: For the URL http://example.com, the resource name is example.com.
- Hostname: The name of the machine on which the resource is saved.
- Filename: The path to the file on the machine.
- Port number: The port number to connect to (typically optional).
- Reference: A reference to a named anchor within a resource, usually identifying a specific location within a file.(typically optional). 

**Performing GET HTTP requests:**
    
    
HTTP stands for HyperText Transfer Protocol and is the basis for data communication on the Internet. The S in https stands for secure, because https communication is usually encrypted. 

What happens when you type a web address into your browser?
<br>
Visiting a website is done by simply sending a http(s) GET request. Pythonistas do this with the `requests` library.

In [ ]:
import requests

url = ('https://newsapi.org/v2/everything?'
       'q=tesla&'
       'sortBy=popularity&'
       'apiKey=' + keyloader.get("newsapi"))

response = requests.get(url)
print(response.json())

This returns a JSON object with the results in an array that we can iterate over. Note that in addition to using the News API, we can also use Python's requests library to retrieve data.

***

Now import the `pandas` package and the `urlretrieve` function from the `urllib.request` sublibrary. Go to [wwww.bundesbank.de](wwww.bundesbank.de) and search for daily term structure on listed federal securities. Then download the `.csv` file "Term structure of interest rates on listed Federal securities (method by Svensson) / residual maturity of 1.0 year / monthly data" `BBSIS.M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A`. 

Inspect the dataset using `read_csv()`. The separator is `;` and decimal is `,`. Set the date to the DataFrame's index, and clean the data. 

Good work so far. Now try plotting the time series. Error? When importing data with `,` notation instead of `.`, Pandas does not recognize that your data is numeric. Replace the comma with a period and set the DataFrame type to float with `.astype(float)`.




In [ ]:
from urllib.request import urlretrieve
import pandas as pd

folder = "data_bundesbank/"
url = 'https://api.statistiken.bundesbank.de/rest/download/BBSIS/M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A?format=csv&lang=en'
#url='https://www.bundesbank.de/statistic-rmi/StatisticDownload?tsId=BBK01.WZ9808'
urlretrieve(url, folder + 'BBSIS.M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A.csv')

#https://api.statistiken.bundesbank.de/rest/download/BBSIS/M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A?format=csv&lang=en

In [ ]:
bundesbank = pd.read_csv('data_bundesbank/BBSIS.M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A.csv', sep=',', decimal=",")
bundesbank = bundesbank.iloc[7:,:2]
bundesbank = bundesbank.rename(columns={"Unnamed: 0":"year_month", "BBSIS.M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A":"interest_rate"})
bundesbank = bundesbank.set_index('year_month')
bundesbank['interest_rate'] = bundesbank['interest_rate'].astype(float)
bundesbank.head(10)

In [ ]:
plt.rcParams['figure.figsize'] = [16, 6]
bundesbank.plot()
plt.show()

***


### Hands-on

1. The Bundesbank offers daily term structure on listed Federal securities. From this [website](https://www.bundesbank.de/en/statistics/money-and-capital-markets/interest-rates-and-yields/daily-term-structure-on-listed-federal-securities-651570) download the term structure of interest rates on listed federal securities with residual maturities of 1.0, 2.0, 3.0, 5.0, and 10.0 years using the techniques you have learned.


2. Sort and clean your data and save it to a Pandas DataFrame.


3. *Homework:* Now construct a 3D yield curve with the dimensions Yield, Period, and Time to Maturity. If you need help, take a look at this great [GitHub repository](https://github.com/plotly/dash-yield-curve) showing a  [3D yield curve](https://www.nytimes.com/interactive/2015/03/19/upshot/3d-yield-curve-economic-growth.html).  

***

## 3. Web scraping 101 <a id='scrape'></a>

**DISCLAIMER:** 
This course material is provided for educational and informational purposes only. It should not be used on websites where automatic scraping is prohibited.

Python makes it easy to import data from the web. But what if we cannot access data through an API or a csv file on a particular website? To overcome this obstacle, we need to learn new techniques, in this case web scraping or web crawling techniques, to obtain both structured and unstructured data from the web. As we progress through this chapter, you will understand the power of Python's web scraping libraries `requests`, `BeautifulSoup`, and `Selenium`. 

In the last few sessions we have been downloading stock quotes from WRDS using the company identifier - the ticker symbol. As you know, the S&P500 index contains the 500 largest U.S. companies and has 500 constituents so far. Of course, if you are interested in a portfolio optimization problem, you need all the stock prices of the S&P500, so you need to create a ticker list to download the data. 

Such a list can be found at [Wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies). Save the table as a Pandas DataFrame. To do this, first `from bs4 import BeautifulSoup` and also import `request`. Request the text of the website URL using `requests.get().text` and save the HTML data to the variable `soup` with `Beautifulsoup(url, 'lxml')`.

In [ ]:
import requests
from bs4 import BeautifulSoup

url  = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text
soup = BeautifulSoup(url,'lxml')

Since the extracted HTML data is structured, we can use BeautifulSoup to parse the data and get the information we are looking for. BeautifulSoup allows us to explore and iterate over specific elements. It already provides many methods (see the [Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)) that can identify elements in the HTML soup for you. Print the title of the HTML container and inspect the `prettify()` function on the soup variable. 

In [ ]:
print(soup.title)

In [ ]:
print(soup.prettify())

Again, look at the source code of the page. The table belongs to the class `wikitable sortable` and can be accessed with `soup.find('table', {'class': 'wikitable sortable'})`. Note that the table has a bode `tbody`. From the table body, we need to iterate over all `tr` elements using BeautifulSoup's `find_all()` method.

In [ ]:
table = soup.find('table',{'class':'wikitable sortable'})

table_body = table.find('tbody')
rows = table_body.find_all('tr')

In [ ]:
rows

Initialize a pandas dataframe and denote the column headers as Wikipedia does. Iterate over all `tr` elements and find all `td` elements. Strip the text of the elements with `text.strip()` and save the data as rows to the dataframe. 

In [ ]:
sp500 = pd.DataFrame(columns = ['Symbol', 'Security','GICS Sector', 'GICS Sub-Industry',
                                'Headquarters Location', 'Date added', 'CIK', 'Founded'])


In [ ]:
for row in rows[1:]:                          
    data = row.find_all('td')                
    data = [element.text.strip() for element in data] # get text
    sp500.loc[len(sp500), :] = data                   # append row to dataframe

Examine the end of the scraped table and filter the dataframe to ensure that it contains only companies in the `Information Technology` GICS sector. 

In [ ]:
sp500.tail(10)

In [ ]:
IT = sp500.loc[sp500['GICS Sector'] == "Information Technology"]
IT

***


### Hands-on

1. Now look at [DAX tickers](https://en.wikipedia.org/wiki/DAX). Save the html document by using `Beautifulsoup`. Go to the [documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) and find out which methods you can apply to your soup object. Print the title and download all links (`hrefs`)


2. Code a web crawler and load [DAX tickers](https://en.wikipedia.org/wiki/DAX) automatically as a Pandas DataFrame in your environment. 


3. Sum up the weights of companies that belong to the industrials sector.

***

## 4. Session Takeaways <a id='takeaway'></a>

*What have we learned in this session?*

- How to use the libraries Requests, Beautifulsoup, and Regex
- How to save source code of a website

*What next?*

Now you know some of the most commonly used web scraping tools in Python. Of course, there are many more, such as the scrapy library, which allows you to compute spiders. Web scraping requires a lot of patience and practice - keep coding!
***